## Importación de paquetes

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d

from numpy import logical_and as npand
from numpy import logical_or as npor
import matplotlib

from sympy import (symbols, pi, I, E, cos, sin, exp, tan, simplify, expand, factor, collect,
                   apart, cancel, expand_trig, diff, Derivative, Function, integrate, limit,
                   series, Eq, solve, dsolve, Matrix, N, preorder_traversal, Float, solve_linear_system,
                   eye, zeros, lambdify, Symbol,hessian, acos)
from sympy.physics.mechanics import dynamicsymbols, init_vprinting
from sympy.physics.mechanics import ReferenceFrame, Point, LagrangesMethod
from sympy import legendre_poly

from functools import lru_cache

## Inicializar espacio de trabajo

In [ ]:
init_vprinting()

In [ ]:
x, t = symbols('x t')

## Probando Polinomios de Legendre

In [ ]:
legendre_poly(20)

In [ ]:
N = 100
prec = 20
for ii in legendre_poly(N, polys = True).real_roots():
    pass
    #print(ii.evalf(n = prec))

In [ ]:
N = 100
prec = 20
for ii in legendre_poly(N, polys = True).diff().real_roots():
    pass
    #print(ii.evalf(n = prec))

## Funciones que generan N collocation points

In [ ]:
@lru_cache
def LG(N, precission = 20):
    return [ii.evalf(n = precission) for ii in legendre_poly(N, polys = True).real_roots()]
@lru_cache
def LGR(N, precission = 20):
    pol = legendre_poly(N, polys = True) + legendre_poly(N-1, polys = True)
    return [ii.evalf(n = precission) for ii in pol.real_roots()]
@lru_cache
def LGL(N, precission = 20):
    root_list = [ii.evalf(n = precission) for ii in legendre_poly(N-1, polys = True).diff().real_roots()]
    return [-1.,] + root_list + [1.,]
@lru_cache
def LGLm(N, precission = 20):
    return LGL(N+2, precission)[1:-1]
@lru_cache
def LG_1(N, precission = 20):
    return [-1] + LG(N-1, precission)
@lru_cache
def LG2(N, precission = 20):
    return [-1]+LG(N-2, precission)+[1]

## Bases de Polinomios de Lagrange (simbólico)

In [ ]:
#### CREDIT: https://gist.github.com/folkertdev/084c53887c49a6248839 ####

from operator import mul
from functools import reduce, lru_cache
from itertools import chain

# sympy symbols
x = symbols('x')

# convenience functions
product = lambda *args: reduce(mul, *(list(args) + [1]))


# this product may be reusable (when creating many functions on the same domain)
# therefore, cache the result
@lru_cache(16)
def lag_pol(labels, j):
    def gen(labels, j):
        k = len(labels)
        current = labels[j]
        for m in labels:
            if m == current:
                continue
            yield (x - m) / (current - m)
    return expand(product(gen(labels, j)))


def lagrangePolynomial(xs, ys):
    # based on https://en.wikipedia.org/wiki/Lagrange_polynomial#Example_1
    k = len(xs)
    total = 0

    # use tuple, needs to be hashable to cache
    xs = tuple(xs)

    for j, current in enumerate(ys):
        t = current * lag_pol(xs, j)
        total += t

    return total

In [ ]:
a, b, c = symbols('a b c')
abcpol = lagrangePolynomial([-1, 0, 1], [a, b, c])
abcpol

In [ ]:
h = symbols('h')

_parab_interp = lagrangePolynomial([0, h/2, h],[a,b,c])
_parab_interp.factor().collect(x)

In [ ]:
@lru_cache
def phi_LGL(N, i):
    return lag_pol(tuple(LGL(N)), i)
@lru_cache
def phi_LG_1(N, i):
    return lag_pol(tuple(LG_1(N)), i)
@lru_cache
def phi_LG2(N, i, precission = 20):
    return lag_pol(tuple(LG2(N, precission)), i)

In [ ]:
jj = phi_LGL(3,0)
jj

In [ ]:
jj.evalf(subs = {x:1})

In [ ]:
def LGL_abcpol(N, precission = 20):
    letters = [symbols(ii) for ii in 'abcdefghijklmnopqrsuvw']
    return lagrangePolynomial(LGL(N, precission), letters[:N])

def LG_1_abcpol(N, precission = 20):
    letters = [symbols(ii) for ii in 'abcdefghijklmnopqrsuvw']
    return lagrangePolynomial(LG_1(N, precission), letters[:N])

def LG2_abcpol(N, precission = 20):
    letters = [symbols(ii) for ii in 'abcdefghijklmnopqrsuvw']
    return lagrangePolynomial(LG2(N, precission), letters[:N])


In [ ]:
@lru_cache
def LG_1_end_p_fun(N, precission = 20):
    coefs = symbols(f'c_0:{N}')
    pol_lag = lagrangePolynomial(LG_1(N, precission), coefs)
    res = pol_lag.subs(x, 1)
    return lambdify(coefs, res)

@lru_cache
def LG_1_diff_end_p_fun(N, precission = 20):
    coefs = symbols(f'c_0:{N}')
    pol_lag = lagrangePolynomial(LG_1(N, precission), coefs)
    res = pol_lag.diff(x).subs(x, 1)
    return lambdify(coefs, res)

## Matrices de Derivación y Barycentric coordinates

In [ ]:
@lru_cache
def get_taus(n, scheme, precission = 20):
    if scheme == 'LGL' or scheme == 'D2':
        taus = LGL(n, precission)
    elif scheme == 'LG':
        taus = LG_1(n, precission)
    elif scheme == 'LG2':
        taus = LG2(n, precission)
    elif scheme == 'LGLm':
        taus = LGL(n, precission)
    return taus

In [ ]:
@lru_cache
def v_coef(n, i, scheme = 'LGL', precission = 20):
    taus = get_taus(n+1, scheme, precission)
    prod_coef = [ii for ii in range(n+1)]
    prod_coef.pop(i)
    v_i = 1.0
    for jj in prod_coef:
        v_i *= (taus[i]-taus[jj])
    return 1./v_i

In [ ]:
@lru_cache
def matrix_D_bary(n, scheme = 'LGL', precission = 20):
    taus = get_taus(n+1, scheme, precission)
    M = zeros(n+1)
    v_arr = [v_coef(n, ii, scheme, precission) for ii in range(n+1)]
    for i in range(n+1):
        j_range = [j for j in range(n+1)]
        j_range.pop(i)
        for j in j_range:
            M[i, j] = (v_arr[j]/v_arr[i])/(taus[i]-taus[j])
    for j in range(n+1):
        M[j,j] = -sum(M[j,:])
    return M

In [ ]:
def v_sum(t_arr, i):
    n = len(t_arr)
    prod_coef = [ii for ii in range(n)]
    prod_coef.pop(i)
    v_i = 1.0
    for jj in prod_coef:
        v_i *= (t_arr[i]-t_arr[jj])
    return 1./v_i

def bary_poly(t_arr, y_arr):
    n = len(t_arr)
    v_arr = [v_sum(t_arr, ii) for ii in range(n)]
    sup = 0
    for i in range(n):
        sup+= v_arr[i]*y_arr[i]/(t-t_arr[i])
    inf = 0
    for i in range(n):
        inf+= v_arr[i]/(t-t_arr[i])
    poly_fun = lambdify([t,], sup/inf)
    
    def new_poly(t):
        t = np.array(t, dtype='float64')
        cond_list = [t == t_i for t_i in t_arr]
        func_list = list(y_arr)
        func_list.append(poly_fun)
        return np.piecewise(t, cond_list, func_list)
        
    return new_poly
    

In [ ]:
matrix_D_bary(3,)

In [ ]:
import casadi as cas

In [ ]:
from optibot.casadi import sympy2casadi

In [ ]:
@lru_cache
def LG_1_end_p_fun_cas(N, precission = 20):
    coefs = symbols(f'c_0:{N}')
    pol_lag = lagrangePolynomial(LG_1(N, precission), coefs)
    res = pol_lag.subs(x, 1)
    x_cas = cas.SX.sym('x', N)
    res_cas = sympy2casadi(res, coefs, cas.vertsplit(x_cas))
    return cas.Function(
    "dynamics_x",
    [x_cas],
    [res_cas])

@lru_cache
def LG_1_diff_end_p_fun_cas(N, precission = 20):
    coefs = symbols(f'c_0:{N}')
    pol_lag = lagrangePolynomial(LG_1(N, precission), coefs)
    res = pol_lag.diff(x).subs(x, 1)
    x_cas = cas.SX.sym('x', N)
    res_cas = sympy2casadi(res, coefs, cas.vertsplit(x_cas))
    return cas.Function(
    "dynamics_x",
    [x_cas],
    [res_cas])

In [ ]:
from optibot.pseudospectral import LG_end_p_fun_cas, LG_diff_end_p_fun_cas

## Problema de ejemplo: bloque que desliza

$$\dot x_1 = x_2$$
$$\dot x_2 = u $$

Minimizar $t_f$ sujeto a:

$t_0 = 0\$

$x_2(0) = x_2(t_f) = 0$

$x_1(0) = 0\ ,\ x_1(t_f) = 1$

$ u_{min} < u(t) < u_{max}$

In [ ]:
LG_1_end_p_fun(3, 20)(*np.array([1,2,3]))

In [ ]:
from scipy.interpolate import CubicHermiteSpline as hermite

@lru_cache
def get_taus(n, scheme, precission = 20):
    if scheme == 'LGL' or scheme == 'D2':
        taus = LGL(n, precission)
    elif scheme == 'LG':
        taus = LG_1(n, precission)
    elif scheme == 'LG2':
        taus = LG2(n, precission)
    elif scheme == 'LGLm':
        taus = LGL(n, precission)
    return taus

def find_der_polyline(x_n, xp, yp):
    n = np.searchsorted(xp, x_n)
    n = np.where(n-1>0, n-1, 0)
    deriv_arr = (yp[1:] - yp[:-1])/(xp[1:] - xp[:-1])
    return deriv_arr[n]

def get_pol_u(scheme, N, uu): 
    if scheme == 'LG2':
        pol_u = bary_poly(LG(N-2), uu)
    elif scheme == 'LG':
        pol_u = bary_poly(LG(N-1), uu)
    elif scheme == 'LGLm':
        pol_u = bary_poly(LGLm(N-2), uu)
    elif scheme == 'LGL' or 'D2':
        pol_u = bary_poly(LGL(N), uu)
    return pol_u

def get_pol_x(scheme, qq, vv, t0, t1):
    N = len(qq)
    tau_x = get_taus(N, scheme)
    qq_d = 2/(t1 - t0) * matrix_D_bary(N-1, scheme)@qq
    vv_d = 2/(t1 - t0) * matrix_D_bary(N-1, scheme)@vv

    pol_q = bary_poly(tau_x, qq)
    pol_v = bary_poly(tau_x, vv)
    pol_q_d = bary_poly(tau_x, qq_d)
    pol_v_d = bary_poly(tau_x, vv_d)
    return pol_q, pol_v, pol_q_d, pol_v_d

def extend_x_arrays(qq, vv, scheme):
    N = len(qq)
    if scheme == 'LG':
        tau_x = LG_1(N) + [1]
        endp_f = LG_1_end_p_fun(N, 20)
        qq_1 = float(endp_f(*qq))
        vv_1 = float(endp_f(*vv))
        qq = np.array(list(qq) + [qq_1,],dtype='float64')
        vv = np.array(list(vv) + [vv_1,],dtype='float64')
    else:
        tau_x = get_taus(N, scheme)
    return tau_x, qq, vv

def extend_u_array(uu, scheme, N):
    tau_u = get_taus(N, scheme)
    if scheme == 'LG2':
        uu = np.array([uu[0]] + list(uu) + [uu[-1]],dtype='float64')
    elif scheme == 'LG':
        tau_u = LG_1(N) + [1]
        uu = np.array([uu[0]] + list(uu) + [uu[-1]],dtype='float64')
    elif scheme == 'LGLm':
        uu = np.array([uu[0]] + list(uu) + [uu[-1]])
    return tau_u, uu

def get_hermite_x(qq, vv, aa, tau_x, t0, t1):
    N = len(qq)
    coll_p = t_0 + (1 + np.array(tau_x,dtype='float64'))*(t1 - t0)/2
    her_q = hermite(coll_p, qq, vv)
    her_v = hermite(coll_p, vv, aa)
    her_q_d = her_q.derivative()
    her_v_d = her_v.derivative()
    return her_q, her_v, her_q_d, her_v_d

def error_transcr(qq, vv, uu, scheme, t0, t1, u_interp = 'pol', x_interp = 'pol', problem='bloq'):
    N = len(qq)
    scheme_opts = ['LG', 'LGL', 'D2', 'LG2', 'LGLm']
    if scheme not in scheme_opts:
        NameError(f'Invalid scheme.\n valid options are {scheme_opts}')
    t_arr = np.linspace(-1, 1, 1000)
    if u_interp == 'pol':
        pol_u = get_pol_u(scheme, N, uu)
        u_arr = pol_u(t_arr)
    elif u_interp == 'lin':
        tau_u, uu = extend_u_array(uu, scheme, N)
        u_arr = np.interp(t_arr, tau_u, uu)
    elif u_interp == 'smooth':
        tau_u, uu = extend_u_array(uu, scheme, N)
        uu_dot = np.gradient(uu, tau_u)
        u_arr = hermite(tau_u, uu, uu_dot)(t_arr)
    else:
        raise NameError('Invalid interpolation method for u.\n valid options are "pol", "lin", "smooth"') 
        
    if x_interp == 'pol':
        tau_x = get_taus(N, scheme)
        pol_q, pol_v, pol_q_d, pol_v_d = get_pol_x(scheme, qq, vv, t0, t1)
        q_arr = pol_q(t_arr)
        v_arr = pol_v(t_arr)
        q_arr_d = pol_q_d(t_arr)
        v_arr_d = pol_v_d(t_arr)
    elif x_interp == 'lin':
        tau_x, qq, vv = extend_x_arrays(qq, vv, scheme)
        q_arr = np.interp(t_arr, tau_x, qq)
        v_arr = np.interp(t_arr, tau_x, vv)
        coll_p = t_0 + (1 + np.array(tau_x,dtype='float64'))*(t1 - t0)/2
        t_arr_lin = np.linspace(t0, t1, 1000)
        q_arr_d = find_der_polyline(t_arr_lin, coll_p, qq)
        v_arr_d = find_der_polyline(t_arr_lin, coll_p, vv)
    elif x_interp == 'Hermite':
        tau_x, qq, vv = extend_x_arrays(qq, vv, scheme)
        if problem == 'bloq':
            aa = uu
        elif problem == 'pend':
            aa = uu - np.sin(qq)
        her_q, her_v, her_q_d, her_v_d = get_hermite_x(qq, vv, aa, tau_x, t0, t1)
        t_arr_lin = np.linspace(t0, t1, 1000)
        q_arr = her_q(t_arr_lin)
        v_arr = her_v(t_arr_lin)
        q_arr_d = her_q_d(t_arr_lin)
        v_arr_d = her_v_d(t_arr_lin)
    else:
        raise NameError('Invalid interpolation method for x.\n valid options are "pol", "lin", "Hermite"') 
    
    if problem == 'bloq':
        err_q = (q_arr_d-v_arr)**2
        err_v = (v_arr_d-u_arr)**2
    elif problem == 'pend':
        err_q = (q_arr_d-v_arr)**2
        err_v = (v_arr_d-(u_arr - np.sin(q_arr)))**2
    else:
        raise NameError('Invalid problem name') 
    
    err_q_num = np.trapz(err_q, t_arr)* (t1-t0)/2
    err_v_num = np.trapz(err_v, t_arr)* (t1-t0)/2
    
    
    return err_q_num, err_v_num, err_q_num + err_v_num 

In [ ]:
import time

In [ ]:
solve_repetitions = 30

#### LGL: función objetivo alternativa

$u(\tau)$ está definida como el polinomio de grado N-1 que pasa por los N valores calculados de $U^N$. Encontrar cómo obtener $\int_{-1}^{1}{u(\tau)^2}$ en función de $U^N$

In [ ]:
def obj_f_u_LGL(N, precission = 20):
    coefs = symbols(f'c_0:{N}')
    pol_lag = lagrangePolynomial(LGL(N, precission), coefs)
    res = integrate(pol_lag ** 2, [x, -1, 1])
    return lambdify(coefs, res)

def obj_f_u_LG2(N, precission = 20):
    coefs = symbols(f'c_0:{N}')
    pol_lag = lagrangePolynomial(LG(N, precission), coefs)
    res = integrate(pol_lag ** 2, [x, -1, 1])
    return lambdify(coefs, res)

### Formulación clásica $\dot x = f(x,u)$

In [ ]:
x_sym = cas.SX.sym('x', 2)
x_dot_sym = cas.SX.sym('x_dot', 2)
u_sym = cas.SX.sym('u',1)

dynam_f_x = cas.Function(
    "dynamics_x",
    [x_sym, x_dot_sym, u_sym],
    [x_dot_sym[1]-u_sym]
)

In [ ]:
N = 7
opti = cas.Opti()
opti.solver('ipopt')

In [ ]:
x_opti = opti.variable(N, 2)
u_opti = opti.variable(N)
tau_arr = LGL(N, 20)
t_f = opti.variable(1)
t_0 = 0
u_lim = opti.parameter(1)

In [ ]:
D_mat = sympy2casadi(matrix_D_bary(N-1, 'LGL',20), [], [])
x_dot_opti = (2/t_f - t_0) * D_mat@x_opti

#cost = cas.sum1(u_opti**2)
#f_obj = obj_f_u_LGL(N)
#cost = f_obj(*cas.vertsplit(u_opti))
cost = t_f
opti.minimize(cost)

In [ ]:
opti.subject_to(x_opti[0,:].T == [0., 0.])
opti.subject_to(x_opti[-1,:].T == [1. ,0.])
opti.subject_to(t_f > 0)

for ii in range(N):
    opti.subject_to(dynam_f_x(x_opti[ii,:], x_dot_opti[ii,:], u_opti[ii,:])==0)
    opti.subject_to(x_opti[ii,1] == x_dot_opti[ii,0])
    opti.subject_to(u_opti[ii] <= u_lim[0])
    opti.subject_to(-u_lim[0] <= u_opti[ii])

opti.set_value(u_lim, 1.)
opti.set_initial(t_f, 1.)

In [ ]:
cput0 = time.time()
for ii in range(solve_repetitions):
    sol = opti.solve()
cput1 = time.time()
cpudt_LGL = (cput1-cput0)/solve_repetitions

In [ ]:
xx_sol_LGL = sol.value(x_opti)
uu_sol_LGL = sol.value(u_opti)
t_f_LGL = sol.value(t_f)

t_arr_LGL = t_0 + (1 + np.array(tau_arr))*(t_f_LGL - t_0)/2

q_pol_LGL = lagrangePolynomial(t_arr_LGL, xx_sol_LGL[:,0])
v_pol_LGL = lagrangePolynomial(t_arr_LGL, xx_sol_LGL[:,1])
u_pol_LGL = lagrangePolynomial(t_arr_LGL, uu_sol_LGL[:])

q_interp_f_LGL = lambdify([x,],q_pol_LGL)
v_interp_f_LGL = lambdify([x,],v_pol_LGL)
u_interp_f_LGL = lambdify([x,],u_pol_LGL)

In [ ]:
t_arr_LGL

In [ ]:
plt.figure(figsize=[12,10])
plt.plot(t_arr_LGL, xx_sol_LGL[:,0], 'ro', label = 'position')
plt.plot(t_arr_LGL, xx_sol_LGL[:,1], 'bo', label = 'speed')
plt.plot(t_arr_LGL, uu_sol_LGL[:], 'go', label = 'control')
N_interp = 300
t_int_arr_LGL = np.linspace(t_0, t_f_LGL, N_interp)
plt.plot(t_int_arr_LGL, q_interp_f_LGL(t_int_arr_LGL), 'r')
plt.plot(t_int_arr_LGL, v_interp_f_LGL(t_int_arr_LGL), 'b')
plt.plot(t_int_arr_LGL, u_interp_f_LGL(t_int_arr_LGL), 'g')
plt.grid()
plt.legend()

In [ ]:
#u_sq_LGL = u_interp_f_LGL(t_int_arr)**2
#quality_LGL = np.trapz(u_sq_LGL, t_int_arr)
quality_LGL = t_f_LGL
quality_LGL

In [ ]:
cpudt_LGL*1000

trun_err_LGL = error_transcr(q_pol_LGL, v_pol_LGL, u_pol_LGL, t_0, t_f_LGL)
trun_err_LGL

### Formulación $D^2$ de Ross, Rea y Fahroo

In [ ]:
q_sym = cas.SX.sym('q', 1)
q_dot_sym = cas.SX.sym('q_dot', 1)
q_dot_dot_sym = cas.SX.sym('q_dot_dot', 1)
u_sym = cas.SX.sym('u',1)

dynam_f_q = cas.Function(
    "dynamics",
    [q_sym, q_dot_sym, q_dot_dot_sym, u_sym],
    [q_dot_dot_sym-u_sym]
)

In [ ]:
N = 9
opti = cas.Opti()
opti.solver('ipopt')

In [ ]:
x_opti = opti.variable(N)
u_opti = opti.variable(N)
tau_arr = LGL(N, 20)
t_f = opti.variable(1)
t_0 = 0
u_lim = opti.parameter(1)

In [ ]:
D_mat = sympy2casadi(matrix_D_bary(N-1, 'D2',20), [], [])
x_dot_opti = (2/t_f - t_0) * D_mat@x_opti
x_dot_dot_opti = (2/t_f - t_0) * D_mat@x_dot_opti

#cost = cas.sum1(u_opti**2)
#f_obj = obj_f_u_LGL(N)
#cost = f_obj(*cas.vertsplit(u_opti))
cost = t_f
opti.minimize(cost)

In [ ]:
opti.subject_to(x_opti[0] == [0.])
opti.subject_to(x_opti[-1] == [1.])
opti.subject_to(x_dot_opti[0] == [0.])
opti.subject_to(x_dot_opti[-1] == [0.])
opti.subject_to(t_f > 0)

for ii in range(N):
    opti.subject_to(dynam_f_q(x_opti[ii], x_dot_opti[ii], x_dot_dot_opti[ii], u_opti[ii,:])==0)
    opti.subject_to(u_opti[ii] <= u_lim[0])
    opti.subject_to(-u_lim[0] <= u_opti[ii])

opti.set_value(u_lim, 1.)
opti.set_initial(t_f, 1.)

In [ ]:
cput0 = time.time()
for ii in range(solve_repetitions):
    sol = opti.solve()
cput1 = time.time()
cpudt_D2 = (cput1-cput0)/solve_repetitions

In [ ]:
xx_sol_D2 = sol.value(x_opti)
uu_sol_D2 = sol.value(u_opti)
vv_sol_D2 = sol.value(x_dot_opti)
t_f_D2 = sol.value(t_f)

t_arr_D2 = t_0 + (1 + np.array(tau_arr))*(t_f_D2 - t_0)/2

q_pol_D2 = lagrangePolynomial(t_arr_D2, xx_sol_D2[:])
v_pol_D2 = lagrangePolynomial(t_arr_D2, xx_sol_D2[:]).diff(x)
u_pol_D2 = lagrangePolynomial(t_arr_D2, uu_sol_D2[:])

q_interp_f_D2 = lambdify([x,],q_pol_D2)
v_interp_f_D2 = lambdify([x,],v_pol_D2)
u_interp_f_D2 = lambdify([x,],u_pol_D2)

In [ ]:
plt.figure(figsize=[12,10])
plt.plot(t_arr_D2, xx_sol_D2[:], 'ro', label = 'position')
plt.plot(t_arr_D2, vv_sol_D2[:], 'bo', label = 'speed')
plt.plot(t_arr_D2, uu_sol_D2[:], 'go', label = 'control')
N_interp = 300
t_int_arr_D2 = np.linspace(t_0, t_f_D2, N_interp)
plt.plot(t_int_arr_D2, q_interp_f_D2(t_int_arr_D2), 'r')
plt.plot(t_int_arr_D2, v_interp_f_D2(t_int_arr_D2), 'b')
plt.plot(t_int_arr_D2, u_interp_f_D2(t_int_arr_D2), 'g')
plt.grid()
plt.legend()

In [ ]:
#u_sq_D2 = u_interp_f_D2(t_int_arr)**2
#quality_D2 = np.trapz(u_sq_D2, t_int_arr)
quality_D2 = t_f_D2
quality_D2

In [ ]:
cpudt_D2*1000

trun_err_D2 = error_transcr(q_pol_D2, v_pol_D2, u_pol_D2, t_0, t_f_D2)
trun_err_D2

## Nuevo esquema LG2

In [ ]:
q_sym = cas.SX.sym('q', 1)
q_dot_sym = cas.SX.sym('q_dot', 1)
q_dot_dot_sym = cas.SX.sym('q_dot_dot', 1)
u_sym = cas.SX.sym('u',1)

dynam_f = cas.Function(
    "dynamics",
    [q_sym, q_dot_sym, q_dot_dot_sym, u_sym],
    [q_dot_dot_sym-u_sym]
)

In [ ]:
N = 7
opti = cas.Opti()
opti.solver('ipopt')

In [ ]:
x_opti = opti.variable(N+2)
u_opti = opti.variable(N)
tau_arr_LG2 = LG2(N+2, 20)
t_f = opti.variable(1)
t_0 = 0
u_lim = opti.parameter(1)

In [ ]:
D_mat = sympy2casadi(matrix_D_bary(N+1, "LG2",20), [], [])
x_dot_opti = (2/t_f - t_0) * D_mat@x_opti
x_dot_dot_opti = (2/t_f - t_0) * D_mat@x_dot_opti

#cost = cas.sum1(u_opti**2)
#f_obj = obj_f_u_LG2(N)
#cost = f_obj(*cas.vertsplit(u_opti))
cost = t_f
opti.minimize(cost)

In [ ]:
opti.subject_to(x_opti[0] == [0.])
opti.subject_to(x_opti[-1] == [1.])
opti.subject_to(x_dot_opti[0] == [0.])
opti.subject_to(x_dot_opti[-1] == [0.])
opti.subject_to(t_f > 0)

for ii in range(1, N+1):
    opti.subject_to(dynam_f_q(x_opti[ii], x_dot_opti[ii], x_dot_dot_opti[ii], u_opti[ii-1,:])==0)
    opti.subject_to(u_opti[ii-1] <= u_lim[0])
    opti.subject_to(-u_lim[0] <= u_opti[ii-1])

opti.set_value(u_lim, 1.)
opti.set_initial(t_f, 1.)

In [ ]:
cput0 = time.time()
for ii in range(solve_repetitions):
    sol = opti.solve()
cput1 = time.time()
cpudt_LG2 = (cput1-cput0)/solve_repetitions

In [ ]:
xx_sol_LG2 = sol.value(x_opti)
uu_sol_LG2 = sol.value(u_opti)
vv_sol_LG2 = sol.value(x_dot_opti)
t_f_LG2 = sol.value(t_f)

t_arr_LG2 = t_0 + (1 + np.array(tau_arr_LG2))*(t_f_D2 - t_0)/2
coll_p_t = t_0 + (1 + np.array(LG(N)))*(t_f_LG2 - t_0)/2

q_pol_LG2 = lagrangePolynomial(t_arr_LG2, xx_sol_LG2[:])
v_pol_LG2 = lagrangePolynomial(t_arr_LG2, xx_sol_LG2[:]).diff(x)
u_pol_LG2 = lagrangePolynomial(coll_p_t, uu_sol_LG2[:])

q_interp_f_LG2 = lambdify([x,],q_pol_LG2)
v_interp_f_LG2 = lambdify([x,],v_pol_LG2)
u_interp_f_LG2 = lambdify([x,],u_pol_LG2)

In [ ]:
plt.figure(figsize=[12,10])
plt.plot(t_arr_LG2, xx_sol_LG2[:], 'ro', label = 'position')
plt.plot(t_arr_LG2, vv_sol_LG2[:], 'bo', label = 'speed')
plt.plot(coll_p_t, uu_sol_LG2[:], 'go', label = 'control')
N_interp = 300
t_int_arr_LG2 = np.linspace(t_0, t_f_LG2, N_interp)
plt.plot(t_int_arr_LG2, q_interp_f_LG2(t_int_arr_LG2), 'r')
plt.plot(t_int_arr_LG2, v_interp_f_LG2(t_int_arr_LG2), 'b')
plt.plot(t_int_arr_LG2, u_interp_f_LG2(t_int_arr_LG2), 'g')
plt.grid()
plt.legend()

In [ ]:
#u_sq_LG2 = u_interp_f_LG2(t_int_arr)**2
#quality_LG2 = np.trapz(u_sq_LG2, t_int_arr)
quality_LG2 = t_f_LG2
quality_LG2

In [ ]:
cpudt_LG2*1000